In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import os

In [46]:
# Dataset from https://www.kaggle.com/datasets/puneet6060/intel-image-classification/ was used.

# Constants
IMAGE_SIZE = 150 # Original image size is 150x150 pixels
BATCH_SIZE = 16
EPOCHS = 10

# Paths
main_directory = 'intel_image_dataset'
train_directory = os.path.join(main_directory, 'seg_train/seg_train')
test_directory = os.path.join(main_directory, 'seg_test/seg_test')
predictions_directory = os.path.join(main_directory, 'seg_pred')

In [47]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=1,
)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_directory,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=1,
)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [48]:
# Mapping training data and validation data
train_data = train_data.map(lambda x, y: (tf.keras.layers.Rescaling(1./255)(x), y))
validation_data = validation_data.map(lambda x, y: (tf.keras.layers.Rescaling(1./255)(x), y))

In [49]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])

In [50]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, epochs=EPOCHS, verbose=1)

In [36]:
loss, accuracy = model.evaluate(validation_data)

188/188 [==============================] - 6s 30ms/step - loss: 0.5736 - accuracy: 0.8040


In [37]:
# Save the model

directory_to_store_model = 'tf_models'

if not os.path.exists(directory_to_store_model):
    os.mkdir(directory_to_store_model)
model.save(os.path.join(directory_to_store_model, 'model.keras'))